In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
le=LabelEncoder()
data['Gender']=le.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [3]:
from sklearn.preprocessing import OneHotEncoder
oh=OneHotEncoder()
oh_df=oh.fit_transform(data[['Geography']])
col=oh.get_feature_names_out(['Geography'])
oh_df=oh_df.toarray()
oh_df=pd.DataFrame(oh_df,columns=col)
oh_df
data=pd.concat([data.drop('Geography',axis=1),oh_df],axis=1)

In [4]:
data
with open('encoder_gender.pkl','wb') as file:
    pickle.dump(le,file)
with open('encoder_geography.pkl','wb') as file:
    pickle.dump(oh,file) 

In [5]:
x=data.drop('Exited',axis=1)
y=data['Exited']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)
with open('scaler.pkl','wb') as file:
    pickle.dump(sc,file) 

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import datetime


In [7]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
]
)

In [8]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer="adam", loss="binary_crossentropy",metrics=['accuracy'])
loss=tf.keras.losses.BinaryCrossentropy()
loss

In [9]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir='logs/fit' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensarflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)
Early_stoppping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)


In [10]:
#training model
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[tensarflow_callback,Early_stoppping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 5ms/step - loss: 0.4443 - accuracy: 0.8125 - val_loss: 0.3917 - val_accuracy: 0.8360
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3846 - accuracy: 0.8424 - val_loss: 0.3577 - val_accuracy: 0.8570
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3547 - accuracy: 0.8544 - val_loss: 0.3450 - val_accuracy: 0.8615
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3439 - accuracy: 0.8586 - val_loss: 0.3462 - val_accuracy: 0.8565
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3398 - accuracy: 0.8579 - val_loss: 0.3428 - val_accuracy: 0.8570
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3369 - accuracy: 0.8608 - val_loss: 0.3407 - val_accuracy: 0.8620
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3331 - accuracy: 0.8627 - val_loss: 0.3442 - val_accuracy: 0.8590

In [11]:
model.save('model.h5')

d:\dlp\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
%load_ext tensorboard

In [13]:
import os

log_dir = 'logs/fit20250613-232044'
print("Log dir exists:", os.path.exists(log_dir))
print("Subfolders:", os.listdir(log_dir))
for sub in os.listdir(log_dir):
    print(f"{sub} contents:", os.listdir(os.path.join(log_dir, sub)))


Log dir exists: True
Subfolders: ['train', 'validation']
train contents: ['events.out.tfevents.1749837044.DESKTOP-R0BGQ46.14180.8.v2']
validation contents: ['events.out.tfevents.1749837045.DESKTOP-R0BGQ46.14180.9.v2']


In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [15]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 8796), started 0:00:58 ago. (Use '!kill 8796' to kill it.)